In [1]:
#this function checks for convenience in the VISTA algorithm
def Frob(Uold,Dsqold,Vold,U,Dsq,V): #the previous and latest results of the singular value decomposition
    denom=np.sum(Dsqold**2) #since this is a line vector, this is fairly straightforward
    utu=replicate_r_lazy_mult_left(Dsq,(U.T @ Uold)) #this just looks like row-wise multiplication...replicated
    vtv=replicate_r_lazy_mult_left(Dsq,(V.T @ Vold))
    uvprod=np.sum(np.diag(utu @ vtv))
    num=denom+np.sum(Dsq**2)-2*uvprod
    num=num/np.max(np.array([denom,1e-9]))
    return num

In [2]:
def replicate_r_lazy_mult_left(a,b): #this function replicates R's unique style of multiplication
    #a must be a 1-D vector, b must be a 2-D matrix
    #print(a)
    #print(b)
    temp1=np.ravel(b,order='F') #unwrap the matrix
    temp2=np.ravel(np.array([a]*int(np.ceil(len(temp1)/len(a)))))
    difference=len(np.ravel(np.array([a]*int(np.ceil(len(temp1)/len(a))))))-len(temp1)
    if difference>=1:
        temp2=temp2[:-(len(temp2)-len(temp1))]
    answer=np.reshape(np.multiply(temp1,temp2),b.shape,order='F')
    return answer
    #answer=b[:,0] #this keeps one column in
    #for i in range(b.shape[1]): #this takes the number of columns of b
        #print(i)
    #    temp=np.diag(np.outer(a,b[:,i]))
    #    answer=np.c_[answer,temp] #thanks, stackoverflow!
    #answer=answer[:,1:]
    #return answer

In [3]:
def replicate_r_lazy_mult_right(b,a): #i hate r
    #b must be a matrix, a must be a 1-D vector
    temp1=np.ravel(b,order='F') #unwrap the matrix
    temp2=np.ravel(np.array([a]*int(np.ceil(len(temp1)/len(a))))) #replicate the 1d array
    difference=len(np.ravel(np.array([a]*int(np.ceil(len(temp1)/len(a))))))-len(temp1)
    if difference>=1:
        temp2=temp2[:-(len(temp2)-len(temp1))]
    answer=np.reshape(np.multiply(temp1,temp2),b.shape,order='F')
    return answer

In [4]:
def replicate_lazy_r_division(b,a): #i hate r
    #b must be a matrix, a must be a 1-D vector
    temp1=np.ravel(b,order='F') #unwrap the matrix
    temp2=np.ravel(np.array([a]*int(np.ceil(len(temp1)/len(a))))) #replicate the 1d array
    difference=len(np.ravel(np.array([a]*int(np.ceil(len(temp1)/len(a))))))-len(temp1)
    if difference>=1:
        temp2=temp2[-(len(temp2)-len(temp1)):]
    answer=np.reshape(np.divide(temp1,temp2),b.shape,order='F')
    return answer

In [5]:
def replicate_r_pmax(a,b):
    #a and b must be 1-D and of the same size
    c=[]
    for i in range(len(a)):
        if a[i]>=b[i]:
            c.append(a[i])
        else:
            c.append(b[i])
            
    return np.array(c)

In [6]:
def normalize_data(TEC,SH):
    #TEC is your m x n x t matrix
    TEC=((TEC**0.1) - 1)/0.1 #box-cox transform
    all_mean=np.mean(TEC[~np.isnan(TEC)])
    all_sd=(np.std(TEC[~np.isnan(TEC)]))
    TEC=(TEC-all_mean)/all_sd
    SH=(SH**0.1-1)/0.1
    SH=(SH-all_mean)/all_sd
    return TEC,SH

In [7]:
def unnormalize_data(impute):
    #this unnormalizes the data
    imputed_map=[]
    for i in range(impute[2]):
        imputed_i=impute[i]
        imputed_i=imputed_i*all_sd+all_mean #reverse the standardization
        imputed_i=(0.1*imputed_i+1)**10 #reverse box-cos
        imputed_map.append(imputed_i)
    return imputed_map

In [8]:
def VISTA(x,SH,l1,l2,l3,r,thresh,maxit,penalty):
    #x and SH are m x n x t matrices - m latitudes, n longitudes, t timesteps
    #must be numpy ndarrays
    #x must have NaN values where there are no observations
    #SH must not have ANY NaN values
    m,n,t=x.shape
    xnas=np.isnan(x) #find out where NaNs are located
    xfill=x #copy x over

    #declare holders
    U=[]
    V=[]
    Dsq=[]
    Impute=[]

    #initialize U,V,Dsq
    for time in range(t):
        u=np.random.randn(m,r)
        v=np.random.randn(n,r)
        temp1,temp2,temp3=np.linalg.svd(u,False) #this is not exactly the same as the R implementation, so more steps are taken here
        temp4,temp5,temp6=np.linalg.svd(v,False)
        U.append(temp1) #could be -temp1 and -temp3, but R gives inconsistent results, so I'm going with non-negatives
        V.append(temp4)
        Dsq.append(np.ones(r))
        Impute.append(U[time]@replicate_r_lazy_mult_left(np.ones(r),V[time].T))

    #hopefully that works
    #moving on
    xfill[xnas]=0 #replace the NaNs with 0
    ratio=np.ones(t)
    ratio_hist=[]
    iter=1
    loss_hist=[]

    #while time
    while((iter<maxit) and (np.max(ratio)>thresh)):
        iter=iter+1
        U_old=U
        V_old=V
        Dsq_old=Dsq
        Impute_old=Impute

        #calculate the current loss
        loss=0
        for time in range(t):
            A_t=U_old[time] @ np.sqrt(Dsq_old[time])
            B_t=V_old[time] @ np.sqrt(Dsq_old[time])
            x_star=Impute_old[time]
            resid_mat=x[:,:,time]-x_star
            resid_mat[xnas[:,:,time]]=0
            loss=loss+np.linalg.norm(resid_mat)+l1*(0 if (penalty=='first' and time>=2) else 1)*np.linalg.norm(A_t)+np.linalg.norm(B_t)+l3*np.linalg.norm(SH[:,:,time]-x_star)
            if (l2!=0 and time>=2):
                loss=loss+l2*np.linalg.norm(x_star-Impute_old[time-1])

        loss_hist.append(loss/2)

        #U step for all time points
        for time in range(t-1):
            fill=xfill[:,:,time]
            fill_na=xnas[:,:,time]
            u=U[time]
            dsq=Dsq[time]
            v=V[time]

            if(t==1):
                ts=0
                const=1+l3
            elif (time==1):
                ts=Impute[time+1]*l2
                const=1+l2+l3
            elif (time==t):
                ts=Impute[time-1]*l2
                const=1+l2+l3
            else:
                ts=(Impute[time+1]+Impute[time-1])*l2
                const=1+2*l2+l3

            B=u.T @ (fill+ts+l3*SH[:,:,time])
            temp1=replicate_r_lazy_mult_right(B,dsq)
            temp2=replicate_lazy_r_division(temp1,const*dsq+l1*(0 if (penalty=='first' and time>=2) else 1))
            #B=B*dsq/(const*dsq+l1*ifel)
            B=temp2
            del temp1
            del temp2
            #sigh
            tempu,tempd,tempv=np.linalg.svd(B.T,False)
            u=u@tempv.T
            dsq=tempd
            v=tempu #this notation i swear...
            xhat=u@replicate_r_lazy_mult_left(dsq,v.T)
            Impute[time]=xhat
            fill[fill_na]=xhat[fill_na]
            U[time]=u
            V[time]=v
            Dsq[time]=dsq
            xfill[:,:,time]=fill

        #V step for all time points
        for time in range(t-1):
            fill=xfill[:,:,time]
            fill_na=xnas[:,:,time]
            u=U[time]
            dsq=Dsq[time]
            v=V[time]

            if(t==1):
                ts=0
                const=1+l3
            elif (time==1):
                ts=Impute[time+1]*l2
                const=1+l2+l3
            elif (time==t):
                ts=Impute[time-1]*l2
                const=1+l2+l3
            else:
                ts=(Impute[time+1]+Impute[time-1])*l2
                const=1+2*l2+l3

            A=np.transpose((fill+ts+l3*SH[:,:,time])@v)
            #A=((fill+ts+l3*SH[:,:,time])@v).T
            temp1=replicate_r_lazy_mult_right(A,dsq)
            temp2=replicate_lazy_r_division(temp1,const*dsq+l1*(0 if (penalty=='first' and time>=2) else 1))
            tempu,tempd,tempv=np.linalg.svd(A.T,False)
            u=tempu
            dsq=tempd
            v=v@tempv.T
            xhat=u@replicate_r_lazy_mult_left(dsq,v.T)
            Impute[time]=xhat
            fill[fill_na]=xhat[fill_na]
            U[time]=u
            V[time]=v
            Dsq[time]=dsq
            xfill[:,:,time]=fill

        for time in range(t):
            ratio[time]=Frob(U_old[time],Dsq_old[time],V_old[time],U[time],Dsq[time],V[time])

        ratio_hist.append(np.max(ratio))

    U_final=[]
    V_final=[]
    Dsq_final=[]
    Impute_final=[]
    for time in range(t):
        u=xfill[:,:,time]@V[time]
        tempu,tempd,tempv=np.linalg.svd(u)
        u=tempu
        dsq=tempd
        v=V[time]@tempv.T
        dsq=replicate_r_pmax(dsq-l1*(1 if(penalty=='first' and time>=2) else 0),np.zeros(len(dsq))) #soft thresholding
        #print(np.sum(dsq>0))
        rout=np.min(np.array([np.sum(dsq>0)+1,r]))
        U_final.append(u[:,0:rout])
        Dsq_final.append(dsq[0:rout])
        V_final.append(v[:,0:rout])
        Impute_final.append(u[:,0:rout]@(replicate_r_lazy_mult_left(dsq[0:rout],v[:,0:rout].T)))

    return U_final,V_final,Dsq_final,Impute_final,iter,ratio,ratio_hist,loss_hist

In [11]:
import sys
import numpy as np
M = 181;
N = 361;
T = 288;
c = 16936387;
A = np.random.randn(M,N,T)
B=A

mask=np.zeros(M*N*T,dtype=bool)
mask[:c] = True
np.random.shuffle(mask)
mask=mask.reshape(M,N,T)
A[mask] = np.nan
sizeof_fmt(sys.getsizeof(A))

'143.6MiB'

In [16]:
A[A<=1e-3]==1e-6

array([False, False, False, ..., False, False, False])

In [10]:
def sizeof_fmt(num, suffix="B"):
    for unit in ("", "Ki", "Mi", "Gi", "Ti", "Pi", "Ei", "Zi"):
        if abs(num) < 1024.0:
            return f"{num:3.1f}{unit}{suffix}"
        num /= 1024.0
    return f"{num:.1f}Yi{suffix}"

In [17]:
U,V,Dsq,Impute,iter,ratio,ratio_hist,loss_hist=VISTA(A,B,l1=1.0,l2=1.0,l3=1.0,r=17,thresh=1e-5,maxit=20,penalty='all')

In [18]:
np.mean(B[:,:,0]-Impute[0])

8.173583594404756e-05

In [21]:
np.where(Impute[0]<=1e-5)

(array([  0,   0,   0, ..., 180, 180, 180]),
 array([  4,   6,   7, ..., 350, 355, 359]))

In [27]:
len(np.where(B[:,:,0]<=1e-5)[0])

32755

In [ ]:
#main function
A=data_with_nans
B=SH_data
U,V,Dsq,Impute,iter,ratio,ratio_hist,loss_hist=VISTA(A,B,l1=1.0,l2=1.0,l3=1.0,r=17,thresh=1e-5,maxit=20,penalty='all')

In [6]:
import numpy as np

In [7]:
tec=np.array([[1,2,np.nan],[4,5,6]])
SH=np.array([[1,2,3],[4,5,6]])

In [8]:
normalize_data(tec,SH)

(array([[-1.62976063, -0.6406754 ,         nan],
        [ 0.41939991,  0.77660256,  1.07443357]]),
 array([[-1.62976063, -0.6406754 , -0.02950738],
        [ 0.41939991,  0.77660256,  1.07443357]]))